# A logistic regressor from scratch, with 3 layers

In [2]:
import numpy as np
from sklearn.datasets import make_moons
from random import uniform

## Define the ReLU, sigmoid, binary cross entropy loss functions

In [3]:
def relu(x):
    return np.maximum(0,x)

def sigmoid(z):
    return 1/(1+np.exp(-z))

def avg_log_loss(y_pred,y):
    epsilon = 1e-15
    y_pred_stable = np.clip(y_pred, epsilon, 1 - epsilon)
    cost = -np.sum(np.log(y_pred_stable)*y + np.log(1-y_pred_stable)*(1-y))/y_pred.shape[1]
    return np.squeeze(cost)


## Define the gradients of the functions, this is required for backpropogation

In [4]:
def relu_backward(dA, Z):
    dZ = dA*(Z>0)
    return dZ

def sigmoid_backward(dA, Z):
    "dA is the gradient of the next layer, we want the overall derivative, so compute via chain rule"
    s = sigmoid(Z)
    dAdZ = s*(1-s)
    return dAdZ*dA

In [5]:
def leaky_relu(Z):
    # np.maximum finds the element-wise max
    return np.maximum(0.01 * Z, Z)

def leaky_relu_backward(dA, Z):
    dZ = dA * (Z > 0) + dA * (Z <= 0) * 0.01
    return dZ

## Define our model. We will have 1 input layer, 2 hidden layers and 1 output layer

In [14]:
class MLP:
    def __init__(self, X_train, Y_train):
        self.X = X_train
        self.Y = Y_train
        self.m = X_train.shape[1]

    def model(self,i,l1,l2,o):
        self.W1 = np.random.randn(l1, i) * np.sqrt(2 / i)
        self.W2 = np.random.randn(l2, l1) * np.sqrt(2 / l1)
        self.W3 = np.random.randn(o, l2) * np.sqrt(2 / l2)

        self.B1 = np.ones((l1,1)) * 0.01
        self.B2 = np.ones((l2,1)) * 0.01
        self.B3 = np.zeros((o,1))

    def forward(self):
        self.Z1 = np.dot(self.W1,self.X) + self.B1
        self.A1 = leaky_relu(self.Z1)
        self.Z2 = np.dot(self.W2,self.A1) + self.B2
        self.A2 = leaky_relu(self.Z2)
        self.Z3 = np.dot(self.W3,self.A2) + self.B3
        self.output = sigmoid(self.Z3)

        return self.output

    def backward(self):
        
        self.dZ3 = self.output - self.Y
        self.dW3 = np.dot(self.dZ3, self.A2.T)/self.m
        self.dB3 = np.sum(self.dZ3, axis=1, keepdims=True)/self.m
        self.dA2 = np.dot(self.W3.T, self.dZ3)
        self.dZ2 = leaky_relu_backward(self.dA2,self.Z2)
        self.dW2 = np.dot(self.dZ2, self.A1.T)/self.m
        self.dB2 = np.sum(self.dZ2, axis=1, keepdims=True)/self.m
        self.dA1 = np.dot(self.W2.T, self.dZ2)
        self.dZ1 = leaky_relu_backward(self.dA1,self.Z1)
        self.dW1 = np.dot(self.dZ1, self.X.T)/self.m
        self.dB1 = np.sum(self.dZ1, axis=1, keepdims=True)/self.m

    def train(self, no_epochs, learning_rate):
        for i in range(no_epochs):
            Y_pred = self.forward()
            cost = avg_log_loss(Y_pred, self.Y)
            self.backward()

            self.W1 -= learning_rate * self.dW1
            self.W2 -= learning_rate * self.dW2
            self.W3 -= learning_rate * self.dW3

            self.B1 -= learning_rate * self.dB1
            self.B2 -= learning_rate * self.dB2
            self.B3 -= learning_rate * self.dB3

            print(f"Epoch {i}: cost {cost}")

    def accuracy(self):
        Y_pred = self.forward()
        correct = (Y_pred > 0.5) == self.Y
        print(f"Accuracy: {np.mean(correct)}")

In [16]:
X, Y = make_moons(n_samples=500, noise=0.2, random_state=42)
X_train = X.T
Y_train = Y.reshape(1, Y.shape[0])

l = MLP(X_train,Y_train)
l.model(2,5,5,1)
l.train(5000,0.1)

X_val, Y_val = make_moons(n_samples=100, noise=0.4, random_state=42)
X_val = X_val.T
Y_val = Y_val.reshape(1,Y_val.shape[0])
l.X = X_val
l.Y = Y_val
l.accuracy()



Epoch 0: cost 0.8383995706331484
Epoch 1: cost 0.8163651199177903
Epoch 2: cost 0.7977531461988958
Epoch 3: cost 0.7817937710250566
Epoch 4: cost 0.7678686117205527
Epoch 5: cost 0.755601765476891
Epoch 6: cost 0.7448184635618638
Epoch 7: cost 0.7353965040875561
Epoch 8: cost 0.7272331148704532
Epoch 9: cost 0.7200683472249911
Epoch 10: cost 0.7136524061184086
Epoch 11: cost 0.7076772290912632
Epoch 12: cost 0.7019890019249356
Epoch 13: cost 0.6966391905128709
Epoch 14: cost 0.691612632526921
Epoch 15: cost 0.6868591851353387
Epoch 16: cost 0.6822786481214096
Epoch 17: cost 0.6777836212024472
Epoch 18: cost 0.6734368800894778
Epoch 19: cost 0.6692168486623413
Epoch 20: cost 0.6650429701788909
Epoch 21: cost 0.6609212609315364
Epoch 22: cost 0.6568243535040987
Epoch 23: cost 0.6528027210577093
Epoch 24: cost 0.6488264474532489
Epoch 25: cost 0.6448977044199755
Epoch 26: cost 0.6409758623157796
Epoch 27: cost 0.6371092553790214
Epoch 28: cost 0.6333134086495584
Epoch 29: cost 0.629544504